# imports


In [9]:
# %pip install seaborn

In [10]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt

from scipy import stats as st


In [11]:
%pip freeze > requirements.txt


Note: you may need to restart the kernel to use updated packages.
